In [17]:
import csv
import os

# Định nghĩa các giới hạn độ dài để dễ thay đổi
GIOI_HAN_INPUT = 40000
GIOI_HAN_OUTPUT = 5000
DO_DAI_CHUOI_NGAN = 80 # Số ký tự hiển thị khi báo lỗi

def them_system_instruction_va_prefix_va_loc_csv(duong_dan_file_csv_input, duong_dan_file_csv_output, duong_dan_file_txt_instruction):
    """
    Đọc nội dung từ file instruction.txt.
    Đọc file CSV, thêm nội dung instruction.txt, sau đó thêm dòng "Page content:"
    và cuối cùng mới tới nội dung gốc của cột 'input:'.
    Kiểm tra độ dài của cột 'input:' (sau khi thêm instruction và prefix).
    Nếu bất kỳ dòng nào có cột 'input:' vượt quá GIOI_HAN_INPUT
    hoặc cột 'output:' vượt quá GIOI_HAN_OUTPUT, dòng đó sẽ bị bỏ qua.
    Các dòng hợp lệ (với instruction và prefix đã thêm) sẽ được ghi vào một file CSV mới.

    Args:
        duong_dan_file_csv_input (str): Đường dẫn đến file CSV gốc.
        duong_dan_file_csv_output (str): Đường dẫn đến file CSV mới sẽ được tạo.
        duong_dan_file_txt_instruction (str): Đường dẫn đến file .txt chứa system instruction.

    Returns:
        tuple: (so_dong_goc, so_dong_vuot_gioi_han, so_dong_ghi_ra_file_moi, co_loi_doc_instruction).
               Trả về (-1, -1, -1, True) nếu có lỗi nghiêm trọng.
    """

    print(f"--- Bắt đầu xử lý file: {duong_dan_file_csv_input} ---")
    so_dong_goc = 0
    so_dong_vuot_gioi_han = 0
    so_dong_ghi_ra_file_moi = 0
    cac_loi_xu_ly_dong = []
    system_instruction = ""
    co_loi_doc_instruction = False
    prefix_content = "\n\nConference data input:\n\n" # Chuỗi sẽ được thêm vào sau instruction

    # Bước 1: Đọc nội dung từ file instruction.txt
    try:
        with open(duong_dan_file_txt_instruction, 'r', encoding='utf-8') as file_instruction:
            system_instruction = file_instruction.read()
        print(f"Đã đọc thành công system instruction từ '{duong_dan_file_txt_instruction}' (độ dài: {len(system_instruction)})")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file instruction tại '{duong_dan_file_txt_instruction}'.")
        co_loi_doc_instruction = True
        # Tiếp tục xử lý CSV nhưng với instruction rỗng
    except Exception as e:
        print(f"Lỗi không xác định khi đọc file instruction: {e}")
        co_loi_doc_instruction = True
        # Tiếp tục xử lý CSV nhưng với instruction rỗng

    # Bước 2: Xử lý file CSV
    try:
        with open(duong_dan_file_csv_input, 'r', encoding='utf-8') as file_csv_in:
            doc_csv = csv.DictReader(file_csv_in)
            ten_cac_cot = doc_csv.fieldnames

            if not ten_cac_cot:
                 print("Lỗi: File CSV trống hoặc không có header.")
                 return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

            if 'input:' not in ten_cac_cot or 'output:' not in ten_cac_cot:
                 print("Lỗi: File CSV phải có cả cột 'input:' và 'output:'.")
                 return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

            print(f"Các cột trong file: {', '.join(ten_cac_cot)}")
            print(f"Giới hạn độ dài kiểm tra (sau khi thêm instruction và prefix): input: > {GIOI_HAN_INPUT}, output: > {GIOI_HAN_OUTPUT}")
            print(f"File đầu ra sẽ là: {duong_dan_file_csv_output}")
            print("-" * 30)

            # Tạo thư mục chứa file output nếu chưa tồn tại
            thu_muc_output = os.path.dirname(duong_dan_file_csv_output)
            if thu_muc_output and not os.path.exists(thu_muc_output):
                os.makedirs(thu_muc_output)
                print(f"Đã tạo thư mục đầu ra: {thu_muc_output}")

            with open(duong_dan_file_csv_output, 'w', encoding='utf-8', newline='') as file_csv_out:
                ghi_csv = csv.DictWriter(file_csv_out, fieldnames=ten_cac_cot)
                ghi_csv.writeheader() # Ghi header vào file mới

                for so_dong_hien_tai, dong in enumerate(doc_csv, start=2): # Bắt đầu từ dòng 2 (sau header)
                    so_dong_goc += 1
                    vuot_gioi_han = False
                    chi_tiet_vuot = [] # Lưu chi tiết cột nào vượt giới hạn

                    try:
                        # Lấy giá trị input gốc và thêm instruction + prefix
                        gia_tri_input_goc = dong.get('input:', '') # Sử dụng get với giá trị mặc định
                        # Nối theo thứ tự: instruction -> prefix -> input gốc
                        gia_tri_input_moi = system_instruction + prefix_content + gia_tri_input_goc

                        # Cập nhật giá trị input trong dictionary của dòng
                        dong['input:'] = gia_tri_input_moi
                        do_dai_input_moi = len(gia_tri_input_moi)

                        # Kiểm tra độ dài input mới
                        if do_dai_input_moi > GIOI_HAN_INPUT:
                            vuot_gioi_han = True
                            chi_tiet_vuot.append(f"input: ({do_dai_input_moi}/{GIOI_HAN_INPUT})")

                        # Kiểm tra độ dài output (không thay đổi)
                        gia_tri_output = dong.get('output:', '')
                        do_dai_output = len(gia_tri_output)
                        if do_dai_output > GIOI_HAN_OUTPUT:
                            vuot_gioi_han = True
                            chi_tiet_vuot.append(f"output: ({do_dai_output}/{GIOI_HAN_OUTPUT})")


                        if vuot_gioi_han:
                            so_dong_vuot_gioi_han += 1
                            print(f"Dòng {so_dong_hien_tai}: Bỏ qua vì vượt giới hạn - {' & '.join(chi_tiet_vuot)}")
                        else:
                            # Dòng hợp lệ, ghi vào file mới (với giá trị input đã cập nhật)
                            ghi_csv.writerow(dong)
                            so_dong_ghi_ra_file_moi += 1

                    except Exception as e:
                         # Xử lý lỗi đọc hoặc ghi dòng cụ thể
                         print(f"Lỗi không xác định khi xử lý dòng {so_dong_hien_tai}: {e}")
                         cac_loi_xu_ly_dong.append(f"Dòng {so_dong_hien_tai}: {e}")
                         # Không tăng so_dong_vuot_gioi_han hoặc so_dong_ghi_ra_file_moi cho dòng lỗi này

            print("-" * 30)
            print("--- Kết thúc xử lý file CSV ---")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại '{duong_dan_file_csv_input}'.")
        return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

    except Exception as e:
        print(f"Lỗi không xác định trong quá trình xử lý file CSV: {e}")
        return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

    if cac_loi_xu_ly_dong:
        print("\nCác lỗi xử lý dòng cụ thể:")
        for loi in cac_loi_xu_ly_dong:
            print(f"- {loi}")

    return (so_dong_goc, so_dong_vuot_gioi_han, so_dong_ghi_ra_file_moi, co_loi_doc_instruction)


# --- Phần ví dụ sử dụng ---
duong_dan_file_goc = "./src/conference/examples/extract_infor_done_300_shrink.csv"       # Thay đổi đường dẫn file CSV gốc của bạn
duong_dan_file_moi = "./src/conference/examples/extract_infor_done_300_filtered_with_instruction_prefix.csv" # Tên file mới sẽ được tạo
duong_dan_instruction = "extract_system_instruction.txt"     # Thay đổi đường dẫn file instruction.txt

so_dong_goc, so_dong_vuot, so_dong_moi, co_loi_instruction = them_system_instruction_va_prefix_va_loc_csv(
    duong_dan_file_goc,
    duong_dan_file_moi,
    duong_dan_instruction
)

print("\n--- Báo cáo kết quả xử lý ---")
if co_loi_instruction:
    print("Cảnh báo: Có lỗi khi đọc file system instruction. Quá trình xử lý CSV tiếp tục nhưng không thêm instruction.")

if so_dong_goc != -1:
    print(f"Tổng số dòng trong file gốc (không bao gồm header): {so_dong_goc}")
    print(f"Số dòng bị loại bỏ (vượt quá giới hạn sau khi thêm instruction và prefix): {so_dong_vuot}")
    print(f"Số dòng được ghi vào file mới '{duong_dan_file_moi}': {so_dong_moi}")
else:
    print("\nKhông thể tạo báo cáo chi tiết do có lỗi nghiêm trọng trong quá trình xử lý.")

--- Bắt đầu xử lý file: ./src/conference/examples/extract_infor_done_300_shrink.csv ---
Đã đọc thành công system instruction từ 'extract_system_instruction.txt' (độ dài: 2976)
Các cột trong file: input:, output:
Giới hạn độ dài kiểm tra (sau khi thêm instruction và prefix): input: > 40000, output: > 5000
File đầu ra sẽ là: ./src/conference/examples/extract_infor_done_300_filtered_with_instruction_prefix.csv
------------------------------
Dòng 127: Bỏ qua vì vượt giới hạn - input: (40261/40000)
Dòng 143: Bỏ qua vì vượt giới hạn - input: (42078/40000)
Dòng 178: Bỏ qua vì vượt giới hạn - input: (41510/40000)
Dòng 231: Bỏ qua vì vượt giới hạn - input: (41882/40000)
------------------------------
--- Kết thúc xử lý file CSV ---

--- Báo cáo kết quả xử lý ---
Tổng số dòng trong file gốc (không bao gồm header): 282
Số dòng bị loại bỏ (vượt quá giới hạn sau khi thêm instruction và prefix): 4
Số dòng được ghi vào file mới './src/conference/examples/extract_infor_done_300_filtered_with_instructi

In [ ]:
import csv
import json
import io
import sys

# --- Phần 1: Tăng giới hạn kích thước trường CSV ---
max_int = sys.maxsize
print("Đang tìm giới hạn kích thước trường CSV tối đa mà hệ thống hỗ trợ...")
while True:
    try:
        csv.field_size_limit(max_int)
        print(f"Đã đặt giới hạn kích thước trường CSV thành: {max_int}")
        break
    except OverflowError:
        max_int = int(max_int / 10)

# --- Phần 2: Đặt tên file và xử lý ---
input_filename = './src/conference/examples/determine_links_train_data.csv' # Tên file CSV đầu vào của bạn
# Đổi tên file output một lần nữa
output_filename = 'output_transformed_final.csv'

print(f"Đang xử lý file: {input_filename}")

try:
    with open(input_filename, 'r', newline='', encoding='utf-8') as infile, \
         open(output_filename, 'w', newline='', encoding='utf-8') as outfile:

        csv_reader = csv.reader(infile)
        csv_writer = csv.writer(outfile)

        try:
            header = next(csv_reader)
            csv_writer.writerow(header)
            print(f"Đã đọc header: {header}")

            for i, row in enumerate(csv_reader):
                current_line_num = i + 2
                if len(row) == 2:
                    input_text = row[0]
                    output_json_str = row[1]
                    transformed_output = ""

                    try:
                        data_dict = json.loads(output_json_str)

                        if isinstance(data_dict, dict):
                            # ---- THAY ĐỔI Ở ĐÂY ----
                            lines = [] # Danh sách để lưu các dòng đã định dạng
                            num_items = len(data_dict) # Lấy số lượng cặp key-value

                            # Sử dụng enumerate để có chỉ số (idx)
                            for idx, (key, value) in enumerate(data_dict.items()):
                                # Tạo dòng cơ bản
                                line = f'"{key}": "{value}"'
                                # Nếu không phải là mục cuối cùng, thêm dấu phẩy
                                if idx < num_items - 1:
                                    line += ","
                                lines.append(line) # Thêm dòng vào danh sách

                            # Nối các dòng trong danh sách bằng ký tự xuống dòng
                            transformed_output = "\n".join(lines)
                            # ---- KẾT THÚC THAY ĐỔI ----
                        else:
                            print(f"Cảnh báo dòng {current_line_num}: Cột output không phải là JSON object: {output_json_str[:100]}...")
                            transformed_output = output_json_str

                    except json.JSONDecodeError:
                        print(f"Cảnh báo dòng {current_line_num}: Không thể parse JSON từ cột output: {output_json_str[:100]}...")
                        transformed_output = output_json_str
                    except Exception as e:
                        print(f"Lỗi xử lý JSON ở dòng {current_line_num}: {e}")
                        transformed_output = output_json_str

                    csv_writer.writerow([input_text, transformed_output])

                else:
                    print(f"Cảnh báo dòng {current_line_num}: Dòng không có đúng 2 cột. Bỏ qua: {row}")

        except csv.Error as e:
            print(f"Lỗi CSV ở dòng {csv_reader.line_num}: {e}")
            print("Kiểm tra lại cấu trúc file CSV hoặc tăng 'field_size_limit' nếu cần.")
        except StopIteration:
             print("File CSV không có dữ liệu sau header.")
        except Exception as e:
             line_num_info = f" (possibly near line {csv_reader.line_num})" if 'csv_reader' in locals() and hasattr(csv_reader, 'line_num') else ""
             print(f"Đã xảy ra lỗi không mong muốn trong quá trình xử lý file{line_num_info}: {e}")

    print(f"\nXử lý hoàn tất. Dữ liệu đã được ghi vào file: {output_filename}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{input_filename}'")
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn bên ngoài quá trình đọc CSV: {e}")

In [ ]:
import csv
import re
import sys

# --- Tăng giới hạn CSV nếu cần ---
max_int = sys.maxsize
print("Đang tìm giới hạn kích thước trường CSV tối đa...")
while True:
    try:
        csv.field_size_limit(max_int)
        print(f"Đã đặt giới hạn kích thước trường CSV thành: {max_int}")
        break
    except OverflowError:
        max_int = int(max_int / 10)

# --- Đặt tên file ---
input_filename = 'output_transformed_final.csv' # File input từ bước trước
# File output mới, tên rõ ràng hơn
output_filename = 'final_truncated_input_at_4.csv'

# --- Biểu thức chính quy để tìm điểm cắt (CHỈ SỐ 4) ---
# Thay đổi ở đây: thay \d+ bằng số 4 cụ thể
pattern_to_find = re.compile(r"\s*4\.\s+Website of ") # Tìm "4. Website of "

print(f"Đang đọc file: {input_filename}")
print(f"Sẽ ghi kết quả vào: {output_filename}")

try:
    with open(input_filename, 'r', newline='', encoding='utf-8') as infile, \
         open(output_filename, 'w', newline='', encoding='utf-8') as outfile:

        csv_reader = csv.reader(infile)
        csv_writer = csv.writer(outfile)

        try:
            # Đọc và ghi header
            header = next(csv_reader)
            csv_writer.writerow(header)
            print(f"Đã đọc header: {header}")

            # Xử lý từng dòng dữ liệu CSV
            for i, row in enumerate(csv_reader):
                current_csv_line_num = i + 2 # Số dòng trong file CSV

                if len(row) == 2:
                    input_text_original = row[0]
                    output_text = row[1] # Giữ nguyên cột output

                    # Tìm vị trí đầu tiên của pattern "4. Website of "
                    match = pattern_to_find.search(input_text_original)

                    if match:
                        # Nếu tìm thấy, lấy vị trí bắt đầu
                        start_index = match.start()
                        # Cắt chuỗi từ đầu đến vị trí đó
                        input_text_cleaned = input_text_original[:start_index].rstrip()
                        # Thêm dấu xuống hàng ở cuối sau khi cắt
                        input_text_cleaned += '\n'
                        # print(f"  (CSV Line {current_csv_line_num}) Đã cắt bỏ từ '4. Website of ' tại vị trí {start_index}.")
                    else:
                        # Nếu không tìm thấy "4. Website of ", giữ nguyên và thêm dấu xuống hàng
                        input_text_cleaned = input_text_original + '\n'


                    # Ghi dòng đã xử lý
                    csv_writer.writerow([input_text_cleaned, output_text])

                else:
                    print(f"Cảnh báo dòng CSV {current_csv_line_num}: Dòng không có đúng 2 cột. Bỏ qua: {row}")

        except csv.Error as e:
            print(f"Lỗi CSV ở dòng {csv_reader.line_num}: {e}")
        except StopIteration:
             print("File CSV không có dữ liệu sau header.")
        except Exception as e:
             line_num_info = f" (possibly near line {csv_reader.line_num})" if 'csv_reader' in locals() and hasattr(csv_reader, 'line_num') else ""
             print(f"Đã xảy ra lỗi không mong muốn trong quá trình xử lý file{line_num_info}: {e}")

    print(f"\nXử lý hoàn tất. Dữ liệu đã được cắt (chỉ tại '4. Website of ') và ghi vào file: {output_filename}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{input_filename}'. Hãy đảm bảo file input có tên này.")
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn bên ngoài quá trình đọc CSV: {e}")

In [6]:
import csv
import os

# Định nghĩa các giới hạn độ dài để dễ thay đổi
GIOI_HAN_INPUT = 34000
GIOI_HAN_OUTPUT = 5000
DO_DAI_CHUOI_NGAN = 80 # Số ký tự hiển thị khi báo lỗi

def xoa_dong_vuot_gioi_han_csv(duong_dan_file_csv_input, duong_dan_file_csv_output):
    """
    Đọc file CSV, kiểm tra độ dài của cột 'input:' và 'output:'.
    Nếu bất kỳ dòng nào có cột 'input:' vượt quá GIOI_HAN_INPUT
    hoặc cột 'output:' vượt quá GIOI_HAN_OUTPUT, dòng đó sẽ bị bỏ qua.
    Các dòng hợp lệ sẽ được ghi vào một file CSV mới.

    Args:
        duong_dan_file_csv_input (str): Đường dẫn đến file CSV gốc.
        duong_dan_file_csv_output (str): Đường dẫn đến file CSV mới sẽ được tạo.

    Returns:
        tuple: (so_dong_goc, so_dong_vuot_gioi_han, so_dong_ghi_ra_file_moi).
               Trả về (-1, -1, -1) nếu có lỗi xảy ra.
    """

    print(f"--- Bắt đầu xử lý file: {duong_dan_file_csv_input} ---")
    so_dong_goc = 0
    so_dong_vuot_gioi_han = 0
    so_dong_ghi_ra_file_moi = 0
    cac_loi_xu_ly = []

    try:
        with open(duong_dan_file_csv_input, 'r', encoding='utf-8') as file_csv_in:
            doc_csv = csv.DictReader(file_csv_in)
            ten_cac_cot = doc_csv.fieldnames

            if not ten_cac_cot:
                 print("Lỗi: File CSV trống hoặc không có header.")
                 return (-1, -1, -1)

            print(f"Các cột trong file: {', '.join(ten_cac_cot)}")
            print(f"Giới hạn độ dài kiểm tra: input: > {GIOI_HAN_INPUT}, output: > {GIOI_HAN_OUTPUT}")
            print(f"File đầu ra sẽ là: {duong_dan_file_csv_output}")
            print("-" * 30)

            # Tạo thư mục chứa file output nếu chưa tồn tại
            thu_muc_output = os.path.dirname(duong_dan_file_csv_output)
            if thu_muc_output and not os.path.exists(thu_muc_output):
                os.makedirs(thu_muc_output)
                print(f"Đã tạo thư mục đầu ra: {thu_muc_output}")


            with open(duong_dan_file_csv_output, 'w', encoding='utf-8', newline='') as file_csv_out:
                ghi_csv = csv.DictWriter(file_csv_out, fieldnames=ten_cac_cot)
                ghi_csv.writeheader() # Ghi header vào file mới

                for so_dong_hien_tai, dong in enumerate(doc_csv, start=2): # Bắt đầu từ dòng 2 (sau header)
                    so_dong_goc += 1
                    vuot_gioi_han = False
                    chi_tiet_vuot = [] # Lưu chi tiết cột nào vượt giới hạn

                    try:
                        # Kiểm tra từng cột cần giới hạn
                        if 'input:' in dong:
                            gia_tri_input = dong.get('input:')
                            do_dai_input = len(gia_tri_input) if gia_tri_input is not None else 0
                            if do_dai_input > GIOI_HAN_INPUT:
                                vuot_gioi_han = True
                                chi_tiet_vuot.append(f"input: ({do_dai_input}/{GIOI_HAN_INPUT})")

                        if 'output:' in dong:
                            gia_tri_output = dong.get('output:')
                            do_dai_output = len(gia_tri_output) if gia_tri_output is not None else 0
                            if do_dai_output > GIOI_HAN_OUTPUT:
                                vuot_gioi_han = True
                                chi_tiet_vuot.append(f"output: ({do_dai_output}/{GIOI_HAN_OUTPUT})")

                        if vuot_gioi_han:
                            so_dong_vuot_gioi_han += 1
                            print(f"Dòng {so_dong_hien_tai}: Bỏ qua vì vượt giới hạn - {' & '.join(chi_tiet_vuot)}")
                        else:
                            # Dòng hợp lệ, ghi vào file mới
                            ghi_csv.writerow(dong)
                            so_dong_ghi_ra_file_moi += 1

                    except Exception as e:
                         # Xử lý lỗi đọc hoặc ghi dòng cụ thể
                         print(f"Lỗi không xác định khi xử lý dòng {so_dong_hien_tai}: {e}")
                         cac_loi_xu_ly.append(f"Dòng {so_dong_hien_tai}: {e}")
                         # Không tăng so_dong_vuot_gioi_han hoặc so_dong_ghi_ra_file_moi cho dòng lỗi này

            print("-" * 30)
            print("--- Kết thúc xử lý file ---")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại '{duong_dan_file_csv_input}'.")
        return (-1, -1, -1)

    except Exception as e:
        print(f"Lỗi không xác định trong quá trình xử lý file: {e}")
        return (-1, -1, -1)

    if cac_loi_xu_ly:
        print("\nCác lỗi xử lý dòng cụ thể:")
        for loi in cac_loi_xu_ly:
            print(f"- {loi}")

    return (so_dong_goc, so_dong_vuot_gioi_han, so_dong_ghi_ra_file_moi)


# --- Phần ví dụ sử dụng ---
duong_dan_file_goc = "./src/conference/examples/extract_infor_done_300_shrink.csv" # Thay đổi đường dẫn file gốc của bạn
duong_dan_file_moi = "./src/conference/examples/extract_infor_done_300_filtered.csv" # Tên file mới sẽ được tạo

so_dong_goc, so_dong_vuot, so_dong_moi = xoa_dong_vuot_gioi_han_csv(duong_dan_file_goc, duong_dan_file_moi)

if so_dong_goc != -1:
    print("\n--- Báo cáo kết quả lọc ---")
    print(f"Tổng số dòng trong file gốc (không bao gồm header): {so_dong_goc}")
    print(f"Số dòng bị loại bỏ (vượt quá giới hạn): {so_dong_vuot}")
    print(f"Số dòng được ghi vào file mới '{duong_dan_file_moi}': {so_dong_moi}")
else:
    print("\nKhông thể tạo báo cáo do có lỗi trong quá trình xử lý.")

--- Bắt đầu xử lý file: ./src/conference/examples/extract_infor_done_300_shrink.csv ---
Các cột trong file: input:, output:
Giới hạn độ dài kiểm tra: input: > 34000, output: > 5000
File đầu ra sẽ là: ./src/conference/examples/extract_infor_done_300_filtered.csv
------------------------------
Dòng 7: Bỏ qua vì vượt giới hạn - input: (35034/34000)
Dòng 11: Bỏ qua vì vượt giới hạn - input: (34639/34000)
Dòng 127: Bỏ qua vì vượt giới hạn - input: (37259/34000)
Dòng 143: Bỏ qua vì vượt giới hạn - input: (39076/34000)
Dòng 178: Bỏ qua vì vượt giới hạn - input: (38508/34000)
Dòng 187: Bỏ qua vì vượt giới hạn - input: (35321/34000)
Dòng 231: Bỏ qua vì vượt giới hạn - input: (38880/34000)
Dòng 241: Bỏ qua vì vượt giới hạn - input: (36852/34000)
Dòng 255: Bỏ qua vì vượt giới hạn - input: (36775/34000)
Dòng 270: Bỏ qua vì vượt giới hạn - input: (34807/34000)
------------------------------
--- Kết thúc xử lý file ---

--- Báo cáo kết quả lọc ---
Tổng số dòng trong file gốc (không bao gồm header): 

In [ ]:
import csv
import json

def kiem_tra_csv_va_json(duong_dan_file_csv, context_size=50):
    """
    Đọc file CSV, kiểm tra xem cột "output:" có phải là JSON hợp lệ hay không,
    và log dòng lỗi ra cùng với context xung quanh vị trí lỗi.

    Args:
        duong_dan_file_csv (str): Đường dẫn đến file CSV.
        context_size (int): Số lượng ký tự trước và sau vị trí lỗi để hiển thị.

    Returns:
        list: Một danh sách các thông báo lỗi. Mỗi thông báo lỗi là một tuple
              (so_dong, cot, thong_tin_loi). Trả về danh sách rỗng nếu không có lỗi.
    """

    cac_loi = []

    try:
        with open(duong_dan_file_csv, 'r', encoding='utf-8') as file_csv:
            doc_csv = csv.DictReader(file_csv)
            for so_dong, dong in enumerate(doc_csv, start=2):  # Bắt đầu từ dòng 2 (bỏ qua header)
                try:
                    input_text = dong.get('input:')
                    output_text = dong.get('output:')

                    # Kiểm tra JSON
                    if output_text is not None:
                        try:
                            json.loads(output_text)
                        except json.JSONDecodeError as e:
                            # Log dòng lỗi ra và context
                            position = e.pos
                            start = max(0, position - context_size)
                            end = min(len(output_text), position + context_size)
                            context = output_text[start:end]

                            print(f"Lỗi JSON ở dòng {so_dong}: {e}")
                            print(f"Vị trí lỗi: {position}")
                            print(f"Context: ...{context}...")
                            cac_loi.append((so_dong, 'output:', f"Lỗi JSON: {e}"))


                except Exception as e:
                    print(f"Lỗi khi xử lý dòng {so_dong}: {e}")
                    cac_loi.append((so_dong, 'Lỗi xử lý dòng', str(e)))  # Thêm thông tin lỗi vào danh sách

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại '{duong_dan_file_csv}'.")
        return [("File", "Không tìm thấy file", duong_dan_file_csv)]  # Trả về một list với thông báo lỗi

    except Exception as e:
        print(f"Lỗi không xác định khi đọc file CSV: {e}")
        return [("File", "Lỗi đọc file", str(e))]

    return cac_loi


# Ví dụ sử dụng
duong_dan_file = './fine_tune_extract_infor.csv'  # Thay đổi đường dẫn file của bạn ở đây
cac_loi = kiem_tra_csv_va_json(duong_dan_file)

if cac_loi:
    print("Các lỗi được tìm thấy:")
    for so_dong, cot, thong_tin_loi in cac_loi:
        print(f"  - Dòng {so_dong}, cột '{cot}': {thong_tin_loi}")
else:
    print("Không có lỗi nào được tìm thấy.")